# 本notebook 是为了应用训练好的XGBoost model
并在水稻GT上做实例进行验证

In [7]:
import numpy as np
import pandas as pd
import pickle
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import joblib
import os
rice_df_activity=pd.read_pickle("./data/GT1/GT1_Rice_df.pkl")
file = open("./model/XGBoost.pkl",'rb')
model=pickle.load(file)
file.close()

In [8]:
X_train = np.load("./data/splits/ecfp/X_train.npy")
y_train = np.load("./data/splits/ecfp/y_train.npy")
model.fit(X_train, y_train.ravel())

/home/hcyao/miniconda3/envs/enzpred/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.4,
              enable_categorical=False, gamma=0, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.016, max_delta_step=0, max_depth=9,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=3800, n_jobs=-1, num_parallel_tree=1,
              objective='binary:logistic', predictor='auto', random_state=42,
              reg_alpha=1.0888187802063715e-06,
              reg_lambda=1.1997406033237056e-07, scale_pos_weight=1,
              subsample=0.55, tree_method='exact', use_label_encoder=True,
              validate_parameters=1, verbosity=0)

In [9]:
def create_input_data(df):
    X = ();
    for ind in df.index:
        emb = df["ESM1b_ts"][ind]
        ecfp = np.array(list(df["ECFP_1024"][ind])).astype(int)           
        X = X +(np.concatenate([ecfp, emb]), );
    return X
rice_test=create_input_data(rice_df_activity)

In [15]:
data_frame = pd.DataFrame(rice_test,index=None)
data_frame.to_csv("./data/Rice_data/rice_features.csv")

In [46]:
test=rice_test[:1]
rice_y=model.predict(test)
rice_y_pro=model.predict_proba(test)

In [43]:
test=()+(rice_test[0])
rice_y=model.predict(test)
rice_y_pro=model.predict_proba(test)

ValueError: operands could not be broadcast together with shapes (0,) (2304,) 

In [45]:
() + (rice_test[0])

ValueError: operands could not be broadcast together with shapes (0,) (2304,) 

In [56]:
rice_df_activity['activity_pre']=rice_y

In [61]:
pro_1=[]
for i in rice_y_pro:
    pro_1.append(i[1])

In [64]:
rice_df_activity['activity_pre_pro']=pro_1

In [66]:
rice_df_activity[rice_df_activity['activity_pre']==1]

,small molecule,enzyme,activity,ECFP_1024,SMILES,ESM1b_ts,activity_pre,activity_pre_pro
0,Baicalein,LOC_Os10g24090.1,NaN,0000000000000000000000000000000000000000000000...,O=c1cc(-c2ccccc2)oc2cc(O)c(O)c(O)c12,"[0.15170568, 0.6114924, -0.69178414, 0.9843046...",1,0.901848
1,Baicalein,LOC_Os06g16000.1,NaN,0000000000000000000000000000000000000000000000...,O=c1cc(-c2ccccc2)oc2cc(O)c(O)c(O)c12,"[0.038845424, 0.44078594, 0.25118843, 0.010497...",1,0.776478
2,Baicalein,LOC_Os09g16090.1,NaN,0000000000000000000000000000000000000000000000...,O=c1cc(-c2ccccc2)oc2cc(O)c(O)c(O)c12,"[0.1073294, 0.47225547, 0.31099978, -0.0448946...",1,0.868863
3,Baicalein,LOC_Os09g16030.1,NaN,0000000000000000000000000000000000000000000000...,O=c1cc(-c2ccccc2)oc2cc(O)c(O)c(O)c12,"[0.10608229, 0.48684025, 0.28982338, 0.0151692...",1,0.819972
4,Baicalein,LOC_Os04g24110.1,NaN,0000000000000000000000000000000000000000000000...,O=c1cc(-c2ccccc2)oc2cc(O)c(O)c(O)c12,"[0.056328245, 0.46821052, 0.28641137, -0.00714...",1,0.840765
...,...,...,...,...,...,...,...,...
12183,Genistein,LOC_Os03g30934.4,NaN,0000000000000000000000000000000000000000000000...,O=c1c(-c2ccc(O)cc2)coc2cc(O)cc(O)c12,"[0.020690762, 0.18643089, 0.47412804, 0.157391...",1,0.557730
12184,Genistein,LOC_Os03g30934.2,NaN,0000000000000000000000000000000000000000000000...,O=c1c(-c2ccc(O)cc2)coc2cc(O)cc(O)c12,"[0.020690762, 0.18643089, 0.47412804, 0.157391...",1,0.557730
12185,Genistein,LOC_Os03g30934.1,NaN,0000000000000000000000000000000000000000000000...,O=c1c(-c2ccc(O)cc2)coc2cc(O)cc(O)c12,"[0.020690762, 0.18643089, 0.47412804, 0.157391...",1,0.557730
12186,Genistein,LOC_Os04g12669.1,NaN,0000000000000000000000000000000000000000000000...,O=c1c(-c2ccc(O)cc2)coc2cc(O)cc(O)c12,"[0.25081506, 0.6468235, -0.7081247, 0.10762866...",1,0.536847


In [70]:
# 使用数据透视函数
rice_df_activity.pivot(index="enzyme",columns="small molecule",values="activity_pre").to_excel("./results/rice/rice_predict.xlsx")

In [71]:
rice_df_activity.pivot(index="enzyme",columns="small molecule",values="activity_pre")

small molecule,"(?)- cis, trans Abscisic acid",(?)-Jasmonic acid,"2,5-dihydroxylbenzoic acid",D-glycerate,Fisetin,Gibberellin A3,Gibberellin A4,GlcNAc,Indole 3-acetate,Kaempferol,...,Umbelliferone,Zeatin,a-ManOBn,a-ManOBn(pNO2),a-ManOBnF5,a-ManOCH2Bn,a-ManOPMP,a-ManOPh,a-ManOPhF5,a-cyano-4-hydroxyl-cinamic acid
enzyme,,,,,,,,,,,,,,,,,,,,,
LOC_Os01g08080.1,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
LOC_Os01g08090.1,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
LOC_Os01g08100.1,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
LOC_Os01g08110.1,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
LOC_Os01g08440.1,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LOC_Os11g25990.1,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
LOC_Os11g26950.1,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
LOC_Os11g27370.1,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
